In [2]:
import struct
import socket
import numpy as np
import torch

def pack_tensor(tensor) -> bytes:
    arr = np.array(tensor, dtype=np.float32)
    shape = arr.shape
    data = arr.tobytes()
    buf = struct.pack('<I', len(shape))
    for d in shape:
        buf += struct.pack('<I', d)
    buf += struct.pack('<I', len(data))
    buf += data
    return buf

def send_four_tensors(tensors):
    assert len(tensors) == 4
    body = struct.pack('<I', 4)           # how many tensors
    for t in tensors:
        body += pack_tensor(t)

    # prefix with total length
    packet = struct.pack('<I', len(body)) + body
    # conn.sendall(packet)
    return packet

In [30]:
imu_tensor = torch.zeros((1, 45, 60))
h = torch.zeros((2, 45, 256))
c = torch.zeros((2, 45, 256))
length = torch.zeros((1))

In [31]:
packet = send_four_tensors([imu_tensor, h, c, length])

In [32]:
packet

b'\x80\xfa\x02\x00\x04\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00-\x00\x00\x00<\x00\x00\x000*\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [14]:
header

b'\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00-\x00\x00\x00<\x00\x00*0'

In [36]:
stream = np.frombuffer(packet, dtype=np.float32)

In [37]:
stream

array([2.73533e-40, 5.60519e-45, 4.20390e-45, ..., 1.40130e-45,
       5.60519e-45, 0.00000e+00], dtype=float32)

In [48]:
out = torch.load("/Users/brianchen/Research/MobilePoser/mobileposer/dev_1746910404.pt")

In [49]:
out["pose"].reshape((out["pose"].shape[0]//72, 72))

tensor([[-0.0367,  0.1219, -0.0107,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0367,  0.1219, -0.0107,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0367,  0.1219, -0.0107,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0225, -0.2737, -0.0241,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0225, -0.2737, -0.0241,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0256, -0.2461, -0.0228,  ...,  0.0000,  0.0000,  0.0000]])

In [50]:
trans = out["tran"].reshape((out["tran"].shape[0]//3, 3))

In [52]:
trans.shape

torch.Size([686, 3])

In [58]:
trans.repeat(45, 3).shape

torch.Size([30870, 9])

In [29]:
s = ','.join(['%g' % v for v in pose[0][1]]) + '#' + \
            ','.join(['%g' % v for v in trans[0]]) + '$'